In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

# 상수 및 초기 조건 (이전과 동일)
e = 1.602e-19
eps_0 = 8.854e-12
k_B = 1.380649e-23
m_e = 9.10938356e-31

V0 = 1000
f0 = 10000
omega = 2 * np.pi * f0

d_gap = 0.01
d_diel = 0.001
eps_r = 4
A = 0.01

n_e0 = 1e15
n_Ar0 = 2.5e25
E_mean0 = 1
Q_diel0 = 0

# Cross section 및 Arrhenius 파라미터 (이전과 동일)
energy_grid = np.linspace(0, 20, 100)
cross_section = 1e-20 * np.exp(-energy_grid/5)

A_arr = 1e-16
E_a = 11.5

def maxwellian(E, E_mean):
    return 2 * np.sqrt(E / (np.pi * E_mean**3)) * np.exp(-E / E_mean)

def rate_coefficient(E_mean):
    max_dist = maxwellian(energy_grid, E_mean)
    return np.trapz(max_dist * cross_section * np.sqrt(2 * e * energy_grid / m_e), energy_grid)

def model(y, t, V0, omega):
    n_e, n_Ar, E_mean, Q_diel = y
    
    # 전압 및 전기장 계산
    V_applied = V0 * np.sin(omega * t)
    V_diel = Q_diel / (eps_0 * eps_r * A / d_diel)
    V_gap = V_applied - V_diel
    E = V_gap / d_gap
    E_abs = np.abs(E)
    
    # 반응률 계산
    k_ion = rate_coefficient(E_mean)
    k_exc = A_arr * np.exp(-E_a / E_mean)
    
    # 미분 방정식
    dn_e_dt = k_ion * n_e * n_Ar - k_exc * n_e * n_Ar
    dn_Ar_dt = -k_ion * n_e * n_Ar
    dE_mean_dt = e * E_abs**2 / (3 * m_e * n_e) - k_exc * n_Ar * E_a
    
    # 유전체 충전/방전
    if E > 0:
        dQ_diel_dt = e * n_e * A * E_abs
    else:
        dQ_diel_dt = -e * n_e * A * E_abs
    
    # 유전체 전하 제한
    Q_max = eps_0 * eps_r * A * V0 / d_diel
    if abs(Q_diel + dQ_diel_dt * 1e-9) > Q_max:  # 1e-9는 시간 스텝 예시
        dQ_diel_dt = 0
    
    return [dn_e_dt, dn_Ar_dt, dE_mean_dt, dQ_diel_dt]

# 시간 그리드
t = np.linspace(0, 5e-4, 10000)  # 시간 범위 확장

# 시뮬레이션 실행
sol = odeint(model, [n_e0, n_Ar0, E_mean0, Q_diel0], t, args=(V0, omega))

# 결과 플로팅
plt.figure(figsize=(15, 12))

plt.subplot(321)
plt.plot(t, sol[:, 0])
plt.title('Electron Density')
plt.xlabel('Time (s)')
plt.ylabel('n_e (m^-3)')

plt.subplot(322)
plt.plot(t, 1 - sol[:, 1]/n_Ar0)
plt.title('Ar Conversion')
plt.xlabel('Time (s)')
plt.ylabel('Conversion')

plt.subplot(323)
plt.plot(t, sol[:, 2])
plt.title('Mean Electron Energy')
plt.xlabel('Time (s)')
plt.ylabel('Energy (eV)')

plt.subplot(324)
plt.plot(t, sol[:, 3])
plt.title('Dielectric Charge')
plt.xlabel('Time (s)')
plt.ylabel('Charge (C)')

plt.subplot(325)
V_applied = V0 * np.sin(omega * t)
V_diel = sol[:, 3] / (eps_0 * eps_r * A / d_diel)
V_gap = V_applied - V_diel
E = V_gap / d_gap
plt.plot(t, E)
plt.title('Electric Field')
plt.xlabel('Time (s)')
plt.ylabel('E (V/m)')

plt.subplot(326)
plt.plot(t, V_applied, label='Applied')
plt.plot(t, V_diel, label='Dielectric')
plt.plot(t, V_gap, label='Gap')
plt.title('Voltages')
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.legend()

plt.tight_layout()
plt.show()